In [4]:
import glob
from PIL import Image
from numpy import expand_dims
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import shutil

# 이진화 전처리 함수

세웅이 모델에 들어가는 이미지 형태는 최종 BGR이여야 함!

In [5]:
# 1, 원본 그대로~
def original(imgpath):
    img = load_img(img_path)
    img_array = img_to_array(img)
    bgr_image = cv2.cvtColor(img_array,cv2.COLOR_RGB2BGR)
    return bgr_image

In [6]:
# adaptive threshold만 적용
def threshold(imgpath):
    # grayscale 변환
    img = load_img(imgpath)
    img = img_to_array(img)
    gray_image = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #(a,b) -> 2차원으로 변환
    gray_image = gray_image.astype('uint8')
    
    # adaptive threshold 적용
    th = cv2.adaptiveThreshold(gray_image,
                               255,
                               cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                               cv2.THRESH_BINARY,
                               99,
                               15)
    
    # bgr 변환
    bgr_image = cv2.cvtColor(th,cv2.COLOR_GRAY2BGR)
    return bgr_image

In [7]:
# morph gradient
def morphology(imgpath):
    # grayscale 변환
    img = load_img(imgpath)
    img = img_to_array(img)
    gray_image = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #(a,b) -> 2차원으로 변환
    gray_image = gray_image.astype('uint8')
    
    # morphgradient 적용
    kernel = np.ones((2,2), np.uint8)
    img_grad = cv2.morphologyEx(gray_image, cv2.MORPH_GRADIENT, kernel)
    
    # bgr로 변환
    bgr_image = cv2.cvtColor(img_grad,cv2.COLOR_GRAY2BGR)
    return bgr_image

In [8]:
# 이미지 대비 향상 -> adaptive threshold
def threshold_sole(imgpath):
    # grayscale 변환
    img = load_img(imgpath)
    img = img_to_array(img)
    gray_image = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #(a,b) -> 2차원으로 변환
    gray_image = gray_image.astype('uint8')
    
    # 이미지 대비를 향상
    image_enhanced = cv2.equalizeHist(gray_image)
    
    # Adaptive Thresholding 적용 
    max_output_value = 255   # 출력 픽셀 강도의 최대값
    neighborhood_size = 99
    subtract_from_mean = 15
    image_binarized = cv2.adaptiveThreshold(image_enhanced,
                                          255,
                                          cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                          cv2.THRESH_BINARY,
                                          99,
                                          15)
    
    # bgr 변환
    bgr_image = cv2.cvtColor(image_binarized,cv2.COLOR_GRAY2BGR)  
    return bgr_image

In [9]:
# morphology -> adaptive threshold
def morph_threshold(imgpath):
    # graysacle
    img = load_img(imgpath)
    img = img_to_array(img)
    gray_image = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    gray_image = gray_image.astype('uint8')
    
    # morph gradient
    kernel = np.ones((2,2), np.uint8)
    img2_grad = cv2.morphologyEx(gray_image, cv2.MORPH_GRADIENT, kernel)
    
    # adaptive threshold
    th = cv2.adaptiveThreshold(img2_grad, 
                               255, 
                               cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                               cv2.THRESH_BINARY,
                               99,
                               15)
    
    # bgr 변환
    bgr_image = cv2.cvtColor(th,cv2.COLOR_GRAY2BGR) # result함수 돌리기 위해 3차원으로 변환 
    return bgr_image

In [10]:
# morph_gradient -> 이미지 대비 향상 -> adaptive threshold
def morph_threshold_sole(imgpath):
    #grayscale
    img = load_img(imgpath)
    img = img_to_array(img)
    gray_image = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    gray_image = gray_image.astype('uint8')
    
    # 이미지 대비를 향상
    image_enhanced = cv2.equalizeHist(gray_image)
    
    # morph_gradient
    kernel = np.ones((2,2), np.uint8)
    img2_grad = cv2.morphologyEx(image_enhanced, cv2.MORPH_GRADIENT, kernel)
    
    # Adaptive Thresholding 적용 
    image_binarized = cv2.adaptiveThreshold(img2_grad,
                                          255,
                                          cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                          cv2.THRESH_BINARY,
                                          99,
                                          15)
    
    # bgr 변환
    bgr_image = cv2.cvtColor(image_binarized,cv2.COLOR_GRAY2BGR)

    
    return bgr_image

# 원본 이미지 경로 불러오기

In [11]:
my_img_pathes = glob.glob("C:/Users/seohyeonpark/프로젝트/TuningStar/*.jpg")

In [12]:
# 경로 확인
my_img_pathes[:5] # '서현' 로컬 컴퓨터에서는 주소 마지막에 \\ 뜸..

['C:/Users/seohyeonpark/프로젝트/TuningStar\\0 (1).jpg',
 'C:/Users/seohyeonpark/프로젝트/TuningStar\\0 (2).jpg',
 'C:/Users/seohyeonpark/프로젝트/TuningStar\\0 (3).jpg',
 'C:/Users/seohyeonpark/프로젝트/TuningStar\\1 (1).jpg',
 'C:/Users/seohyeonpark/프로젝트/TuningStar\\1 (2).jpg']

In [13]:
# 주소 정상적으로 불러지면 이 셀은 건너 뛰어도 됨
i = 0
for path in my_img_pathes:
    path = path.replace('\\','/')
    my_img_pathes[i] = path
    i += 1

In [14]:
my_img_pathes[:5]

['C:/Users/seohyeonpark/프로젝트/TuningStar/0 (1).jpg',
 'C:/Users/seohyeonpark/프로젝트/TuningStar/0 (2).jpg',
 'C:/Users/seohyeonpark/프로젝트/TuningStar/0 (3).jpg',
 'C:/Users/seohyeonpark/프로젝트/TuningStar/1 (1).jpg',
 'C:/Users/seohyeonpark/프로젝트/TuningStar/1 (2).jpg']

# 분류할 카테고리 넘버 리스트, 딕셔너리 만들기
#### category_nums, category_count_dict

In [15]:
category_nums=[]
for img_path in my_img_pathes:
    num_jpg = img_path.split("/")[-1] # 휠 파일 이름.jpg
    category_num = num_jpg.split('.')[0] # jpg 떼어 버리기!
    try:
        category_num = int(category_num) # (i)번 째 안되어 있으면 바로 정수형으로 바꿔잇!
    except:
        if "(" in category_num: # (i)번째 표시되어있으면 날리고 앞에 카테고리만 저장
            category_num = int(category_num.split("(")[0])
        elif "-" in category_num: # - i 로 표시한 사람것도 날리고 앞에 카테고리만 저장
            category_num = int(category_num.split('-')[0])
    category_nums.append(category_num)

각 카테고리 번호 별로 몇 개씩 들어있는 지 카운팅한 뒤, 카테고리(key):등장횟수(value) 형태로 딕셔너리 생성

In [16]:
category_count_dict = {}
for i in range(len(category_nums)):
    num_count = category_nums.count(i)
    category_count_dict[i] = num_count

In [18]:
value = category_count_dict.values()

dict_values([3, 2, 2, 1, 3, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 2, 1, 6, 1, 2, 1, 4, 1, 1, 1, 1, 4, 2, 1, 5, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 3, 3, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 4, 1, 1, 1, 3, 1, 9, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 1, 7, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 4, 8, 1, 1, 7, 3, 1, 2, 1, 1, 1, 1, 1, 2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 10, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 4, 4, 2, 4, 7, 3, 2, 2, 3, 3, 6, 2,

In [23]:
count_list = []
for value in category_count_dict.values():
    if value not in count_list:
        count_list.append(value)

In [24]:
print(count_list) # 2, 3, 5, 7 -> 2100

[3, 2, 1, 6, 4, 5, 9, 7, 8, 10, 0]


Keras의 ImageDataGenerator를 통해 Data Augmentation을 할 수 있는 generator생성.

In [120]:
## data augmentation
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

##ImageDataGenerator에 원하는 옵션들을 추가해주면 옵션들 적용하면서 랜덤하게 생성해줍니다.
datagen = ImageDataGenerator(
        rotation_range = 90,
        width_shift_range = 0.2,
        height_shift_range = 0.2,
        brightness_range = [0.05 , 2.0],
        vertical_flip = True,
        horizontal_flip = True,
        fill_mode= 'nearest')

# 각 전처리 방법으로 이진화 후 다른 폴더에 저장

배정받은 이미지 경로들을 통해 이미지를 가져오고,
하나의 카테고리를 하나의 폴더로 만듦. 위에서도 이야기했듯이 파일 번호 = 카테고리이므로 이미 카테고리 폴더가 생성되어있는 경우는 넘어가게 햇음.

In [150]:
# 하나의 이미지에 대해 파일 이름, 카테고리 인덱스, 같은 카테고리로 분류된 이미지의 개수 반환해주는 함수
def count_category(img_path, category_count_dict):
    # 1. 파일 이름(img_name)
    num_jpg = img_path.split("/")[-1] # 이미지 경로 마지막 (휠 번호.jpg) 만 떼어내기
    category_num = num_jpg.split('.')[0] # .jpg떼어 버리기! (휠 번호.jpg) -> (휠 번호)
    img_name = category_num
    
    # 2. 카테고리 인덱스(category_num)
    try:
        '''
        카테고리 번호만 category_num에 저장하도록 한다 -> 나중에 augmentation한 폴더 이름 설정할 때 사용
        현재 img_name은 1 or 1 (1) or 1-1 형식으로 되어있음.
        맨 앞에 카테고리 분류한 숫자에만 관심이 있기 때문에
        먼저 1과 같이 같은 카테고리 내에 한 개의 이미지만 있는 경우는 int(category_num)으로 숫자 추출,
        에러가 발생한다면 "(", ")","-" 등이 category_num에 포함되어 있는 것이기 때문에,
        각 문장부호 들로 split 후 맨 앞에 카테고리 분류 숫자를 반환
        '''
        category_num = int(category_num) # (i)번 째 안되어 있으면 바로 정수형으로 바꿔잇!
    except:
        if "(" in category_num: # (i)번째 표시되어있으면 날리고 앞에 카테고리만 저장
            category_num = int(category_num.split("(")[0])
        elif "-" in category_num: # - i 로 표시한 사람것도 날리고 앞에 카테고리만 저장
            category_num = int(category_num.split('-')[0])
    
    # 3. 같은 카테고리로 분류된 이미지의 개수(category_count)
    category_count = category_count_dict[category_num] # 해당 이미지의 카테고리에 몇개의 이미지가 있는가
    return img_name, category_count, category_num

# 이미지 전처리 함수
#: 앞서 정해준 이진화 방법들 중 하나를 function에 넣어주면 이미지를 해당 방법으로 이진화 후 
def preprocess(img_path, function):
    img = function(img_path) # function에 전처리 함수를 넣어 해당 방법으로 전처리한다.
    x = expand_dims(img,0) # 차원 추가
    return x

In [151]:
# j == 14 도  j/category_count로 들어가야 할 듯
# 원본이미지로만 구성된 validation set도 만들기
# 

''' 전처리 함수 메뉴
1. original : 원본 이미지
2. threshold : Adaptive만 적용
3. morphology : morph gradient만 적용
4. threshold_sole : 이미지 대비 향상 -> Adaptive
5. morph_threshold : morph gradient -> Adaptive 
6. morph_threshold_sole : 이미지 대비 향상 morph gradient -> adaptive threshold 
'''

# 전처리 함수 리스트
functions = [original,
             threshold,
             #morphology, (정확하게는 이진화하는 방법이 아니라 사용하지 않아도 될 듯?)
             threshold_sole,
             morph_threshold,
             morph_threshold_sole]

for img_path in my_img_pathes:
    
    ''' category별 개수 구하기 '''
    img_name, category_count, category_num = count_category(img_path, category_count_dict)
    
    ''' 전처리 '''
    i = 1
    for function in functions:
        x = preprocess(img_path, function)
        
        img_num = 1500 # int(10000/category_count)
        aug_num = int(img_num/category_count)
        
        if img_num%category_count != 0: # 여기서 print의 결과로 나오는 이미지들은 10000의 약수가 아니므로, augmentation한 결과가 정확히 10000장이 되지 못함. 해당 카테고리에서는 수동으로 이미지 복붙해서 10000장 맞춰주세요.
            print(category_num)
        
        # 각 카테고리 별로 augmentation 이미지 14개 씩 validation 폴더에 저장
        j=0
        
        # 아래 폴더 이름은 "C:/Users/seohyeonpark/프로젝트" 부분만 원하는 경로로 바꿔주세요. 
        validation_folder = "C:/Users/seohyeonpark/프로젝트/Validation_" + str(i) + "/" + str(category_num)
        if not(os.path.isdir(validation_folder)):
            os.makedirs(validation_folder)
        for batch in datagen.flow(x, batch_size=32, save_to_dir=validation_folder, save_prefix=j, save_format="jpeg"):
            j+=1
            if j == 14:
                break

        # 14개 이후에는 augmentation 폴더에 저장
        # 아래 폴더 이름은 "C:/Users/seohyeonpark/프로젝트" 부분만 원하는 경로로 바꿔주세요. 
        folder_path = "C:/Users/seohyeonpark/프로젝트/Data Augmentation_" + str(i) + "/" + str(category_num) # folder_path는 자기가 저장할 경로에 맞게 수정해주어야 함. str(category)는 건들지 말고 앞부분만 수정.
        if not(os.path.isdir(folder_path)):
            os.makedirs(folder_path)
        for batch in datagen.flow(x, batch_size=32, save_to_dir=folder_path, save_prefix=j, save_format="jpeg"):
            j+=1
            if j > aug_num:
                break
                
        i += 1

100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106
106


KeyboardInterrupt: 

# 이후에는 세웅이 모델 학습 붙이기

In [76]:
'''
from keras.preprocessing.image import ImageDataGenerator
from keras import applications

img_width, img_height = 224, 224 ##프로젝트 활용 이미지의 크기
train_data_dir = 'C:/Users/seohyeonpark/프로젝트/Data Augmentation'
datagen = ImageDataGenerator(rescale=1. / 255)
batch_size = 10
# build the ResNet50 Network
model = applications.ResNet50(include_top=False, weights='imagenet')
# Shuffle을 false로 두어야 원래 순서대로 들어감(카테고리가 뒤섞이지 않는다.)
generator = datagen.flow_from_directory(
  train_data_dir,
  target_size=(img_width, img_height),
  batch_size=batch_size,
  class_mode=None,
  shuffle=False)
bottleneck_features_train = model.predict_generator(
  generator)`
# np.save / np.load : 경로 지정한 곳으로 저장, 불러오기 가능, .npy확장자로 save하면 batch로 묶인 벡터들이 들어간다.
np.save('C:/Users/seohyeonpark/프로젝트/feature vector/bottleneck_features_train.npy',
      bottleneck_features_train)
'''

C:\Users\seohyeonpark\Anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 89s 1us/step
Found 48266 images belonging to 756 classes.



KeyboardInterrupt: 